Load the dataset

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf


In [4]:
import csv
import os

datapath = 'clothing-dataset/'
data = []
with open(os.path.join(datapath, 'images.csv')) as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    for line in reader:
        data.append(line)
    

In [5]:
print(data[:10])

[['image', 'sender_id', 'label', 'kids'], ['4285fab0-751a-4b74-8e9b-43af05deee22', '124', 'Not sure', 'False'], ['ea7b6656-3f84-4eb3-9099-23e623fc1018', '148', 'T-Shirt', 'False'], ['00627a3f-0477-401c-95eb-92642cbe078d', '94', 'Not sure', 'False'], ['ea2ffd4d-9b25-4ca8-9dc2-bd27f1cc59fa', '43', 'T-Shirt', 'False'], ['3b86d877-2b9e-4c8b-a6a2-1d87513309d0', '189', 'Shoes', 'False'], ['5d3a1404-697f-479f-9090-c1ecd0413d27', '138', 'Shorts', 'False'], ['b0c03127-9dfb-4573-8934-1958396937bf', '138', 'Shirt', 'False'], ['4c8f245e-a039-46fd-a6b9-1bb51e83fc05', '226', 'T-Shirt', 'False'], ['c995c900-693d-4dd6-8995-43f3051ec488', '337', 'Pants', 'False']]


In [6]:
labels = data[0]
data = data[1:]

In [7]:
print(labels)
print(data[:10])

['image', 'sender_id', 'label', 'kids']
[['4285fab0-751a-4b74-8e9b-43af05deee22', '124', 'Not sure', 'False'], ['ea7b6656-3f84-4eb3-9099-23e623fc1018', '148', 'T-Shirt', 'False'], ['00627a3f-0477-401c-95eb-92642cbe078d', '94', 'Not sure', 'False'], ['ea2ffd4d-9b25-4ca8-9dc2-bd27f1cc59fa', '43', 'T-Shirt', 'False'], ['3b86d877-2b9e-4c8b-a6a2-1d87513309d0', '189', 'Shoes', 'False'], ['5d3a1404-697f-479f-9090-c1ecd0413d27', '138', 'Shorts', 'False'], ['b0c03127-9dfb-4573-8934-1958396937bf', '138', 'Shirt', 'False'], ['4c8f245e-a039-46fd-a6b9-1bb51e83fc05', '226', 'T-Shirt', 'False'], ['c995c900-693d-4dd6-8995-43f3051ec488', '337', 'Pants', 'False'], ['bf78527f-0734-44fd-a968-f6c32c12d61e', '175', 'Shoes', 'False']]


In [8]:
num_examples = len(data)
print(len(data))

5403


In [9]:
label_counts = dict()
for row in data:
    if row[2] not in label_counts:
        label_counts[row[2]] = 0
    label_counts[row[2]] += 1
    
valid_labels = dict()
for k, v in label_counts.items():
    if v >= 100 and k not in ['Not sure', 'Others', 'Skip']:
        valid_labels[k] = v

filename_to_label = dict()
cleaned_data = []
valid_filenames = []
for i in range(len(data)):
    if data[i][2] in valid_labels.keys():
        cleaned_data.append(data[i])
        valid_filenames.append(data[i][0])
        filename_to_label[data[i][0]] = data[i][2]



num_examples = len(cleaned_data)
print(num_examples)

4961


In [39]:
import os
import shutil

imagespath = 'clothing-dataset/images'
images = [f for f in os.listdir(imagespath) if os.path.isfile(os.path.join(imagespath, f))]

num_images_moved = 0



for i in range(len(images)):
    filename = images[i].split(".")[0]
    if filename not in valid_filenames:
        continue
        
    if num_images_moved < num_train + num_validate:
        folder_name = 'train'
    else:
        folder_name = 'test'
    
    new_folder = os.path.join(imagespath, folder_name)
    
    classname = filename_to_label[filename]
    finalfoldername = os.path.join(new_folder, classname)
    
    if not os.path.exists(finalfoldername):
        os.makedirs(finalfoldername)
    
    old_image_path = os.path.join(imagespath, images[i])
    new_image_path = os.path.join(finalfoldername, images[i])
    shutil.move(old_image_path, new_image_path)
    num_images_moved += 1

In [43]:
num_train = 3600
num_validate = 900
num_test = 461
epochs = 15
batch_size = 16
train_dir = os.path.join(os.getcwd(), os.path.join(imagespath, 'train'))
img_height = 128
img_width = 128
num_classes = len(valid_labels)
print(num_classes)

14


In [44]:
import random
seed = random.randint(1, 10000)
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="training",
  seed=seed,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="validation",
  seed=seed,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4500 files belonging to 14 classes.
Using 3600 files for training.
Found 4500 files belonging to 14 classes.
Using 900 files for validation.


In [18]:
model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [19]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [20]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/15
225/225 [==============================] - 42s 185ms/step - loss: 2.6665 - accuracy: 0.2831 - val_loss: 1.9309 - val_accuracy: 0.4344
Epoch 2/15
225/225 [==============================] - 42s 185ms/step - loss: 1.6110 - accuracy: 0.5178 - val_loss: 1.8048 - val_accuracy: 0.4822
Epoch 3/15
225/225 [==============================] - 42s 185ms/step - loss: 0.8557 - accuracy: 0.7503 - val_loss: 2.2285 - val_accuracy: 0.4856
Epoch 4/15
225/225 [==============================] - 42s 186ms/step - loss: 0.3427 - accuracy: 0.9025 - val_loss: 2.6812 - val_accuracy: 0.4722
Epoch 5/15
225/225 [==============================] - 42s 186ms/step - loss: 0.1447 - accuracy: 0.9608 - val_loss: 3.3321 - val_accuracy: 0.5022
Epoch 6/15
225/225 [==============================] - 42s 188ms/step - loss: 0.0843 - accuracy: 0.9833 - val_loss: 3.4140 - val_accuracy: 0.5133
Epoch 7/15
225/225 [==============================] - 42s 186ms/step - loss: 0.0701 - accuracy: 0.9886 - val_loss: 3.8468 - val_ac

In [16]:
test_dir = os.path.join(os.getcwd(), os.path.join(imagespath, 'test'))
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 461 files belonging to 14 classes.


In [45]:
transfer_model = tf.keras.Sequential()
resnet50 = tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(128,128,3),
                   pooling='avg',classes=num_classes,
                   weights='imagenet')

for layer in resnet50.layers:
    layer.trainable=False
    
transfer_model.add(resnet50)

transfer_model.add(Flatten())
transfer_model.add(Dense(128, activation='relu'))
transfer_model.add(Dense(num_classes))

transfer_model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
history = transfer_model.fit(train_ds, validation_data=val_ds, epochs=epochs)


Epoch 1/15
225/225 [==============================] - 68s 298ms/step - loss: 1.3285 - accuracy: 0.5919 - val_loss: 0.9304 - val_accuracy: 0.6856
Epoch 2/15
225/225 [==============================] - 67s 297ms/step - loss: 0.6879 - accuracy: 0.7756 - val_loss: 0.8554 - val_accuracy: 0.7211
Epoch 3/15
225/225 [==============================] - 67s 298ms/step - loss: 0.4711 - accuracy: 0.8425 - val_loss: 0.8861 - val_accuracy: 0.7200
Epoch 4/15
225/225 [==============================] - 67s 297ms/step - loss: 0.3237 - accuracy: 0.8978 - val_loss: 0.9193 - val_accuracy: 0.7233
Epoch 5/15
225/225 [==============================] - 67s 297ms/step - loss: 0.2056 - accuracy: 0.9394 - val_loss: 0.8896 - val_accuracy: 0.7400
Epoch 6/15
225/225 [==============================] - 67s 297ms/step - loss: 0.1374 - accuracy: 0.9653 - val_loss: 0.9435 - val_accuracy: 0.7367
Epoch 7/15
225/225 [==============================] - 67s 298ms/step - loss: 0.0857 - accuracy: 0.9850 - val_loss: 1.0543 - val_ac

In [38]:
history.history

{'loss': [1.3239964246749878,
  0.6962459087371826,
  0.47621896862983704,
  0.3189353346824646,
  0.20820467174053192,
  0.1368134766817093],
 'accuracy': [0.5922222137451172,
  0.7752777934074402,
  0.8394444584846497,
  0.8930555582046509,
  0.9375,
  0.9641666412353516],
 'val_loss': [0.9726228713989258,
  0.9693282842636108,
  0.9378651976585388,
  0.9523977041244507,
  1.0433592796325684,
  1.1192824840545654],
 'val_accuracy': [0.6877777576446533,
  0.695555567741394,
  0.7222222089767456,
  0.7244444489479065,
  0.7166666388511658,
  0.7400000095367432]}